In [ ]:
!pip install apyori

In [ ]:
from google.colab import drive
import pandas as pd
from apyori import apriori
import time

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
ecommerce = pd.read_csv("/content/drive/MyDrive/Project Skripsi/data.csv")
ecommerce.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [ ]:
ecommerce.dropna(inplace=True)

In [ ]:
ecommerce.drop_duplicates(inplace=True)

In [ ]:
ecommerce.isnull().sum()

TransactionNo    0
Date             0
ProductNo        0
ProductName      0
Price            0
Quantity         0
CustomerNo       0
Country          0
dtype: int64

In [ ]:
ecommerce.duplicated().sum()

0

In [ ]:
ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531095 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  531095 non-null  object 
 1   Date           531095 non-null  object 
 2   ProductNo      531095 non-null  object 
 3   ProductName    531095 non-null  object 
 4   Price          531095 non-null  float64
 5   Quantity       531095 non-null  int64  
 6   CustomerNo     531095 non-null  float64
 7   Country        531095 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.5+ MB


In [ ]:
ecommerce.drop(['Date', 'ProductNo', 'Price','Quantity','CustomerNo','Country'], axis=1, inplace=True)
ecommerce.head()

,TransactionNo,ProductName
0,581482,Set Of 2 Wooden Market Crates
1,581475,Christmas Star Wish List Chalkboard
2,581475,Storage Tin Vintage Leaf
3,581475,Tree T-Light Holder Willie Winkie
4,581475,Set Of 4 Knick Knack Tins Poppies


In [ ]:
ecommerce_sample = ecommerce[['TransactionNo']].drop_duplicates().sample(n=15000, random_state=42)
ecommerce_sample_df = pd.merge(ecommerce_sample, ecommerce[['TransactionNo', 'ProductName']], on='TransactionNo', how='left')
ecommerce_sample_df

,TransactionNo,ProductName
0,565446,Heart Filigree Dove Large
1,565446,Filigree Heart Bird White
2,565446,Zinc Metal Heart Decoration
3,565446,Peace Wooden Block Letters
4,565446,Vintage Christmas Gift Sack
...,...,...
343303,577125,3 White Choc Morris Boxed Candles
343304,577125,Classic Chrome Bicycle Bell
343305,577125,3 Rose Morris Boxed Candles
343306,C579277,Heart Of Wicker Large


In [ ]:
product_count=ecommerce_sample_df[['TransactionNo','ProductName']].groupby('ProductName',as_index=False)['TransactionNo'].count(
    ).sort_values(by='TransactionNo',ascending=False)
product_count.rename(columns = {'TransactionNo':'count'},inplace=True)
product_count.head()

,ProductName,count
815,Cream Hanging Heart T-Light Holder,1542
2620,Regency Cakestand 3 Tier,1455
1656,Jumbo Bag Red Retrospot,1407
2202,Party Bunting,1132
1824,Lunch Bag Red Retrospot,1034


In [ ]:
association = ecommerce_sample_df.groupby(['TransactionNo'], as_index=True)['ProductName'].apply(lambda x: list(x))

transactions = association.values.tolist()

transactions[:10]

[['Cream Hanging Heart T-Light Holder',
  'White Moroccan Metal Lantern',
  'Cream Cupid Hearts Coat Hanger',
  'Knitted Union Flag Hot Water Bottle',
  'Red Woolly Hottie White Heart',
  'Set 7 Babushka Nesting Boxes',
  'Glass Star Frosted T-Light Holder'],
 ['Jam Making Set With Jars',
  'Red Coat Rack Paris Fashion',
  'Yellow Coat Rack Paris Fashion',
  'Blue Coat Rack Paris Fashion'],
 ['Bath Building Block Word'],
 ['Alarm Clock Bakelike Pink',
  'Alarm Clock Bakelike Red',
  'Alarm Clock Bakelike Green',
  'Panda And Bunnies Sticker Sheet',
  'Stars Gift Tape',
  'Inflatable Political Globe',
  'Vintage Heads And Tails Card Game',
  'Set/2 Red Retrospot Tea Towels',
  'Round Snack Boxes Set Of4 Woodland',
  'Spaceboy Lunch Box',
  'Lunch Box I Love London',
  'Circus Parade Lunch Box',
  'Charlotte Bag Dolly Girl Design',
  'Red Toadstool Led Night Light',
  'Set 2 Tea Towels I Love London',
  'Vintage Seaside Jigsaw Puzzles',
  'Mini Jigsaw Circus Parade',
  'Mini Jigsaw Space

In [ ]:
start_time = time.time()
association_rules = apriori(transactions,
                            min_support=0.02,
                            min_confidence=0.3,
                            min_lift=5,
)
association_results = list(association_rules)
end_time = time.time()
process_time = end_time - start_time
print("Time taken to process:", process_time, "detik")

Time taken to process: 0.9877221584320068 detik


In [ ]:
results = []
for item in association_results:
    antecedents = ', '.join(map(str, item.ordered_statistics[0].items_base))
    consequents = ', '.join(map(str, item.ordered_statistics[0].items_add))
    support = str(item[1])[:7]
    confidence = str(item[2][0][2])[:7]
    lift = str(item[2][0][3])[:7]

    rows = (antecedents, consequents, support, confidence, lift)
    results.append(rows)

columns = ['Antecedents', 'Consequents', 'Support', 'Confidence', 'Lift']
associations_dataframe = pd.DataFrame(results, columns=columns)
associations_dataframe

,Antecedents,Consequents,Support,Confidence,Lift
0,Alarm Clock Bakelike Green,Alarm Clock Bakelike Red,0.02833,0.62592,13.1495
1,Alarm Clock Bakelike Pink,Alarm Clock Bakelike Red,0.02126,0.60302,12.6685
2,Charlotte Bag Pink Polkadot,Red Retrospot Charlotte Bag,0.02333,0.68627,15.2505
3,Charlotte Bag Suki Design,Red Retrospot Charlotte Bag,0.02226,0.56802,12.6228
4,Charlotte Bag Suki Design,Woodland Charlotte Bag,0.021,0.53571,14.0977
5,Red Hanging Heart T-Light Holder,Cream Hanging Heart T-Light Holder,0.02193,0.67695,6.75154
6,Dolly Girl Lunch Box,Spaceboy Lunch Box,0.02346,0.64351,16.1146
7,Gardeners Kneeling Pad Cup Of Tea,Gardeners Kneeling Pad Keep Calm,0.02346,0.69291,17.2940
8,Green Regency Teacup And Saucer,Pink Regency Teacup And Saucer,0.02846,0.60225,16.9490
9,Green Regency Teacup And Saucer,Regency Cakestand 3 Tier,0.02293,0.48519,5.05406


In [ ]:
associations_dataframe['Support'] = associations_dataframe['Support'].apply(lambda x: float('.'.join(x.split('.'))))
average_Support = associations_dataframe['Support'].mean()

print("Rata-rata Support:", average_Support)

Rata-rata Support: 0.024908387096774197


In [ ]:
associations_dataframe['Confidence'] = associations_dataframe['Confidence'].apply(lambda x: float('.'.join(x.split('.'))))
average_Confidence = associations_dataframe['Confidence'].mean()
print("Rata-rata confidence:", average_Confidence)

Rata-rata confidence: 0.6087509677419355


In [ ]:
associations_dataframe['Lift'] = associations_dataframe['Lift'].apply(lambda x: float('.'.join(x.split('.'))))
average_Lift = associations_dataframe['Lift'].mean()
print("Rata-rata Lift:", average_Lift)

Rata-rata Lift: 10.962398387096773
